In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_27523/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [5]:
analize = Analizer(0.9)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_9_8_9,0.997025,0.992350,0.987563,0.990428,0.000557,0.001201,0.001494,0.001339,"Hidden Size=[25], regularizer=0.01, learning_r..."
1,model_9_8_10,0.997016,0.991863,0.986523,0.989718,0.000558,0.001277,0.001619,0.001438,"Hidden Size=[25], regularizer=0.01, learning_r..."
2,model_9_8_8,0.996998,0.992795,0.988664,0.991137,0.000562,0.001131,0.001362,0.001240,"Hidden Size=[25], regularizer=0.01, learning_r..."
3,model_9_8_11,0.996982,0.991352,0.985548,0.989021,0.000565,0.001357,0.001736,0.001536,"Hidden Size=[25], regularizer=0.01, learning_r..."
4,model_9_8_12,0.996928,0.990835,0.984638,0.988346,0.000575,0.001438,0.001846,0.001630,"Hidden Size=[25], regularizer=0.01, learning_r..."
...,...,...,...,...,...,...,...,...,...,...
381,model_18_5_8,0.982505,0.968260,0.982510,0.974305,0.003273,0.010571,0.004618,0.007770,"Hidden Size=[45], regularizer=0.1, learning_ra..."
382,model_14_8_3,0.982469,0.977493,0.982813,0.981302,0.003280,0.002428,0.004457,0.003383,"Hidden Size=[35], regularizer=0.1, learning_ra..."
385,model_9_4_0,0.982383,0.987049,0.996452,0.992128,0.003296,0.001776,0.000530,0.001190,"Hidden Size=[25], regularizer=0.01, learning_r..."
398,model_18_5_7,0.981886,0.968671,0.984539,0.975378,0.003389,0.010434,0.004082,0.007445,"Hidden Size=[45], regularizer=0.1, learning_ra..."
